In [44]:
import pandas as pd
import numpy as np

# Load data
csv_file_path = "/Users/balmeru/Downloads/AI.csv"
df = pd.read_csv(csv_file_path, low_memory=False)

# Convert rdq to datetime
df['rdq'] = pd.to_datetime(df['rdq'], errors='coerce')

# Fill missing values in txdbq and dvpq
df['txdbq'] = df['txdbq'].fillna(0)
df['dvpq'] = df['dvpq'].fillna(0)

# Create boolean flags
df['income_good'] = ~df['ibq'].isna()
df['se1_good'] = ~df['teqq'].isna()
df['se2_good'] = ~df['ceqq'].isna() & ~df['pstkq'].isna()
df['se3_good'] = ~df['atq'].isna() & ~df['ltq'].isna()

# Filter the dataframe to get relevant rows for calculations
df_filtered = df[(df['fyr_change_dummy'] == 1) & (df['income_good']) & (df['se1_good'] | df['se2_good'] | df['se3_good'])]

# Initialize the count of rows with no previous reports
total_no_previous_reports_count = 0

# Function to calculate the difference in quarters between two fiscal periods
def quarter_diff(fyearq1, fqtr1, fyearq2, fqtr2):
    return (fyearq1 - fyearq2) * 4 + (fqtr1 - fqtr2)

# Iterate over each group by 'tic'
for tic, group in df.groupby('tic'):
    group = group.sort_values(by=['fyearq', 'fqtr'])
    
    # Iterate over each row in the group
    for idx, row in group.iterrows():
        if row['fyr_change_dummy'] == 1 and idx in df_filtered.index:
            # Find the most recent previous report with rdq
            previous_rows = group[(group['rdq'] < row['rdq']) & (group['rdq'].notna())].sort_values(by='rdq')
            
            if previous_rows.empty:
                total_no_previous_reports_count += 1
                # Print the row where no previous report with 'rdq' is found
                print("Row index:", idx)
                print(row)
                print()
                print("=" * 50)

print(f"Total number of rows where no previous report with 'rdq' is found for tickers: {total_no_previous_reports_count}")


Row index: 646
GVKEY                    13027
datadate            1992-06-30
fyearq                    1992
fqtr                         1
fyr                          3
indfmt                    INDL
consol                       C
popsrc                       D
datafmt                    STD
tic                      0097A
ajexq                      1.0
curcdq                     USD
datacqtr                1992Q2
datafqtr                1992Q1
rdq                        NaT
atq                    119.601
ceqq                    69.038
cshoq                   10.556
dvpq                       0.0
ibq                      1.402
lseq                   119.601
ltq                     50.563
pstkq                      0.0
teqq                       NaN
txdbq                      0.0
dvpy                       0.0
costat                       I
mkvaltq                    NaN
prccq                     10.0
market_cap              105.56
lower_bound             84.448
upper_bound            1

In [35]:
nan_median_distance_count = df[(df['fyr_change_dummy'] == 1)  ].shape[0]
print(f"Number of NaN median_distance values when fyr_change_dummy is 1  : {nan_median_distance_count}")

Number of NaN median_distance values when fyr_change_dummy is 1  : 2188


In [36]:
nan_median_distance_count = df[(df['fyr_change_dummy'] == 1)  & (df['median_distance'].notna()) ].shape[0]

print(f"Number of NaN median_distance values when fyr_change_dummy is 1  & median_distance not NaN : {nan_median_distance_count}")


Number of NaN median_distance values when fyr_change_dummy is 1  & median_distance not NaN : 2157


In [37]:

nan_median_distance_count = df[(df['fyr_change_dummy'] == 1)  & (df['median_distance'].notna()) & (df['income_good']) & (df['se1_good'] | df['se2_good'] | df['se3_good'])].shape[0]

print(f"Number of NaN median_distance values when fyr_change_dummy is 1  & median_distance not NaN 'income_good' & 'se good : {nan_median_distance_count}")




Number of NaN median_distance values when fyr_change_dummy is 1  & median_distance not NaN 'income_good' & 'se good : 1668


In [38]:

nan_median_distance_count = df[(df['fyr_change_dummy'] == 1)  & (df['expected_date'].notna()) & (df['income_good']) & (df['se1_good'] | df['se2_good'] | df['se3_good'])].shape[0]

print(f"Number of NaN median_distance values when fyr_change_dummy is 1  & median_distance not NaN 'income_good' & 'se good : {nan_median_distance_count}")




Number of NaN median_distance values when fyr_change_dummy is 1  & median_distance not NaN 'income_good' & 'se good : 1273


In [30]:
1668-1273

395

####  We are losing 413 entries when calculating expected_date while 'fyr_change_dummy' == 1 and median distance were present
#### The reason is because previous rdq or annoucnement date of the last previous report date was not found 

In [43]:
import pandas as pd
import numpy as np

# Load data
csv_file_path = "/Users/balmeru/Downloads/AI.csv"
df = pd.read_csv(csv_file_path, low_memory=False)

# Convert rdq to datetime
df['rdq'] = pd.to_datetime(df['rdq'], errors='coerce')

# Fill missing values in txdbq and dvpq
df['txdbq'] = df['txdbq'].fillna(0)
df['dvpq'] = df['dvpq'].fillna(0)

# Create boolean flags
df['income_good'] = ~df['ibq'].isna()
df['se1_good'] = ~df['teqq'].isna()
df['se2_good'] = ~df['ceqq'].isna() & ~df['pstkq'].isna()
df['se3_good'] = ~df['atq'].isna() & ~df['ltq'].isna()

# Filter the dataframe to get relevant rows for calculations
df_filtered = df[(df['fyr_change_dummy'] == 1) & (df['income_good']) & (df['se1_good'] | df['se2_good'] | df['se3_good'])]

# Initialize expected_date column in the original dataframe
df['expected_date'] = np.nan

# Initialize the count of rows with no previous reports
total_no_previous_reports_count = 0

# Function to calculate the difference in quarters between two fiscal periods
def quarter_diff(fyearq1, fqtr1, fyearq2, fqtr2):
    return (fyearq1 - fyearq2) * 4 + (fqtr1 - fqtr2)

# Iterate over each group by 'tic'
for tic, group in df.groupby('tic'):
    group = group.sort_values(by=['fyearq', 'fqtr'])
    
    # Iterate over each row in the group
    for idx, row in group.iterrows():
        if row['fyr_change_dummy'] == 1 and idx in df_filtered.index:
            # Find the most recent previous report with rdq
            previous_rows = group[(group['rdq'] < row['rdq']) & (group['rdq'].notna())].sort_values(by='rdq')
            
            if previous_rows.empty:
                total_no_previous_reports_count += 1
                # Print the row where no previous report with 'rdq' is found
                print(row)

print(f"Total number of rows where no previous report with 'rdq' is found for tickers: {total_no_previous_reports_count}")


GVKEY                    13027
datadate            1992-06-30
fyearq                    1992
fqtr                         1
fyr                          3
indfmt                    INDL
consol                       C
popsrc                       D
datafmt                    STD
tic                      0097A
ajexq                      1.0
curcdq                     USD
datacqtr                1992Q2
datafqtr                1992Q1
rdq                        NaT
atq                    119.601
ceqq                    69.038
cshoq                   10.556
dvpq                       0.0
ibq                      1.402
lseq                   119.601
ltq                     50.563
pstkq                      0.0
teqq                       NaN
txdbq                      0.0
dvpy                       0.0
costat                       I
mkvaltq                    NaN
prccq                     10.0
market_cap              105.56
lower_bound             84.448
upper_bound            126.672
quarterl

In [42]:
# df.to_csv('expected_dates_output.csv', index=False)
